In [15]:
import numpy as np
import pandas as pd
import setuptools.dist
import plotly.graph_objects as go
import plotly.express as px
from keras import layers as L
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import regularizers, Adam

In [16]:
df = pd.read_csv("../data/people_data.csv")
print(df.head())

   age  ethnicity  gender                        img_name  \
0    1          2       0  20161219203650636.jpg.chip.jpg   
1    1          2       0  20161219222752047.jpg.chip.jpg   
2    1          2       0  20161219222832191.jpg.chip.jpg   
3    1          2       0  20161220144911423.jpg.chip.jpg   
4    1          2       0  20161220144914327.jpg.chip.jpg   

                                              pixels  
0  129 128 128 126 127 130 133 135 139 142 145 14...  
1  164 74 111 168 169 171 175 182 184 188 193 199...  
2  67 70 71 70 69 67 70 79 90 103 116 132 145 155...  
3  193 197 198 200 199 200 202 203 204 205 208 21...  
4  202 205 209 210 209 209 210 211 212 214 218 21...  


In [17]:
df["pixels"] = df["pixels"].apply(lambda x: np.array(x.split(),dtype="float32"))
df["pixels"] = df["pixels"] / 255

In [18]:
x_train = np.zeros(shape=(23705,2304))

for indexing in range(len(df["pixels"])):
    x_train[indexing] = df["pixels"][indexing]

In [19]:
x_train = x_train.reshape(-1,48,48,1)

In [20]:
x = x_train
y = df.iloc[:, :-1].values

In [21]:
x_train,x_test,y_train,y_test = train_test_split(x, y, train_size=0.9, random_state=42, shuffle=True)

In [28]:
early_stopper = tf.keras.callbacks.EarlyStopping(
    monitor="loss",
    patience=5,
    mode="min"
    )

checkpoint_model = tf.keras.callbacks.ModelCheckpoint(
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=False,
    filepath="./.keras"
    )

In [ ]:
COMPILE_OPTIMIZER = Adam()
COMPILE_LOSS = "msle"
COMPILE_METRICS = ["accuracy"]
INPUT_SHAPE = (xTrain.shape[1],xTrain.shape[2],xTrain.shape[3])
OUTPUT_CLASS = 3